In [ ]:
%load_ext autoreload
%autoreload 2
# "%matplotlib widget" is slightly better, but sometimes doesn't work
# "%matplotlib notebook" or "%matplotlib inline" can be used as alternatives
%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
# Debug: show location of imported remote_control code
import remote_control
remote_control.__file__

# Configure acquisition area

In [ ]:
from remote_control import acquisition

## calibration_positions is used to calculate the Z coordinate for each pixel
calibration_positions = [
         (422, 4252, -4.4), # Top-left
         (-10252, 4187, -94.5), # Top-right
         (29, 42684, 122.5), # Bottom-left
         (-10536, 42666, 43) # Bottom-right
]

## target_positions is used to optionally select the target area, if it's different from the calibration positions.
## This is optional and can just be set to calibration_positions if they're the same positions
target_positions = calibration_positions
# target_positions = [
#          (400, 2000),
#          (-10000, 2000),
#          (400, 20000),
#          (-10000, 20000)
# ]

ra = acquisition.RectangularAquisition(
    config_fn = 'remote_config.json', 
)

# Set up safety bounds (optional - comment this out if they're unwanted)
ra.set_image_bounds(
    min_x=-13900,
    max_x=3700,
    min_y=-450,
    max_y=48000,
)


## Use x_pitch/y_pitch to generate the grid with a specific pitch:
ra.generate_targets(
    calibration_positions, 
    target_positions, 
    x_pitch=100,
    y_pitch=100,
#     interpolate_xy=True,  # Use interpolate_xy to allow trapezoidal grids
#     x_size=200, # Use x_size/y_size to generate grid with a specific size:
#     y_size=500,
)

## Use an image mask if there are areas to avoid
# ra.apply_image_mask('rect_acquisition_mask.png')

# Display a preview
ra.plot_targets()

# Write coordinates to file

In [ ]:
OUTPUT_DIR = 'D:\\imagingMS\\2021_08\\your name\\'
IMZML_PREFIX = OUTPUT_DIR + '01052019_Mouse_DHB_pos_mz200-800_px50x50_LR'

In [ ]:
# Write debug coordinates files
ra.write_imzc_coords_file(OUTPUT_DIR + 'positions_imzc.txt')
ra.write_json_coords_file(OUTPUT_DIR + 'positions.json')

# Run acquisition

In [ ]:
ra.acquire(
    filename=IMZML_PREFIX, # Prefix for output coordinates file used in ImzML conversion
    dummy=True, # False - send commands to MALDI, True - don't connect, just print commands
    measure=False, # False - move stage only, True - move stage & acquire data
    email_on_success='test@example.com', # Set to None to suppress
    email_on_failure='test@example.com',  # Set to None to suppress
)

# Cleanup
After imaging run the following cell to terminate Telnet

In [ ]:
import remote_control.control as rc
rc.close(quit=True)